In [8]:
import re
import nltk
import pyspark 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, ArrayType, StringType
from pyspark.sql.functions import col, sum, length, when, count, explode, udf, expr
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem import WordNetLemmatizer

from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [19]:
spark = SparkSession.builder\
	.master("local")\
	.appName("Classification")\
	.config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "2g") \
	.getOrCreate()

24/12/10 20:12:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
df = spark.read.parquet("hdfs://localhost:9000/final/cleaned_data/")
df.show()
df.printSchema()

ERROR:root:KeyboardInterrupt while sending command.>                (0 + 0) / 1]
Traceback (most recent call last):
  File "/home/hungnd/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/hungnd/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

24/12/10 20:16:31 ERROR Executor: Exception in task 0.0 in stage 123.0 (TID 249)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0

In [17]:
df = df.repartition(8)

# Chuyển đổi Sentiment (nếu cần)
indexer = StringIndexer(inputCol="Sentiment", outputCol="label")
data = indexer.fit(df).transform(df)

# Vector hóa từ (CountVectorizer)
vectorizer = CountVectorizer(inputCol="lemmatized_words", outputCol="raw_features")

# Tính IDF để lấy đặc trưng quan trọng
idf = IDF(inputCol="raw_features", outputCol="features")

# Chia dữ liệu train/test
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)


In [12]:
# Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=20)

# Pipeline
lr_pipeline = Pipeline(stages=[vectorizer, idf, lr])

# Train Logistic Regression model
lr_model = lr_pipeline.fit(train_data)

# Đánh giá trên tập test
lr_predictions = lr_model.transform(test_data)

# Đánh giá hiệu suất
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lr_predictions)
print(f"Logistic Regression Accuracy: {lr_accuracy:.2f}")


24/12/10 18:56:30 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/10 18:56:47 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 18:57:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/10 18:57:04 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 18:58:49 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 19:00:21 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 19:01:55 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 19:03:29 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 19:05:07 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 19:06:43 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 19:08:19 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 19:09:54 WARN DAGSchedul

Logistic Regression Accuracy: 0.76


In [13]:
lr_model.save("hdfs://localhost:9000/models/lr_sentiment_model")

24/12/10 19:35:40 WARN TaskSetManager: Stage 42 contains a task of very large size (5076 KiB). The maximum recommended task size is 1000 KiB.
24/12/10 19:35:41 WARN TaskSetManager: Stage 46 contains a task of very large size (4189 KiB). The maximum recommended task size is 1000 KiB.
24/12/10 19:35:42 WARN TaskSetManager: Stage 50 contains a task of very large size (2099 KiB). The maximum recommended task size is 1000 KiB.


In [15]:
from pyspark.ml.pipeline import PipelineModel

# Load Logistic Regression model
loaded_lr_model = PipelineModel.load("hdfs://localhost:9000/models/lr_sentiment_model")

# Dự đoán với dữ liệu mới
new_predictions = loaded_lr_model.transform(test_data)
new_predictions.select("SentimentText", "label", "prediction").show(5, truncate=False)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lr_predictions)
print(f"Logistic Regression Accuracy: {lr_accuracy:.2f}")

24/12/10 19:37:31 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB


+----------------------------------------------------------------------------------------------------------------------+-----+----------+
|SentimentText                                                                                                         |label|prediction|
+----------------------------------------------------------------------------------------------------------------------+-----+----------+
|omgaga im sooo  im gunna cry ive been at this dentist since  i was suposed  just get a crown put on mins              |1.0  |1.0       |
|jb isnt showing in australia any more                                                                                 |1.0  |1.0       |
|lt this is the way i feel right now                                                                                   |1.0  |0.0       |
|cant be bothered i wish i could spend the rest of my life just sat here and going to gigs seriously                   |1.0  |1.0       |
|really dont like doing my room it

24/12/10 19:37:35 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB


Logistic Regression Accuracy: 0.76


In [ ]:
# Random Forest Classifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50)

# Pipeline
rf_pipeline = Pipeline(stages=[vectorizer, idf, rf])

# Train Random Forest model
rf_model = rf_pipeline.fit(train_data)

# Evaluate on test set
rf_predictions = rf_model.transform(test_data)

# Evaluate
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")

24/12/10 20:08:05 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/10 20:08:14 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/10 20:08:37 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 20:08:37 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
24/12/10 20:08:48 WARN DAGScheduler: Broadcasting large task binary with size 9.5 MiB
24/12/10 20:10:16 WARN DAGScheduler: Broadcasting large task binary with size 1033.7 KiB
24/12/10 20:10:17 WARN DAGScheduler: Broadcasting large task binary with size 10.5 MiB
24/12/10 20:10:18 WARN MemoryStore: Not enough space to cache rdd_337_0 in memory! (computed 272.9 MiB so far)
24/12/10 20:10:18 WARN BlockManager: Persisting block rdd_337_0 to disk instead.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/hungnd/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    respo

KeyboardInterrupt: 